# V-1 Bestum rétt

<img src="bestum_rett.png" style="width:200px;">

### 1. Inngangur
Lýstu samhengi verkefnisins: Hvaða vandamál er verið að leysa? Skýrðu hvað, hvers vegna og
hvernig verkefnið er unnið.

Næring er gríðarlega mikilvæg fyrir manneskjuna en fólk á það til að líta framhjá hollu fæði því það telur óhollari matinn þægilegri og ódýrari. Markmið verkefnisins er því að búa til máltíð sem uppfyllir ráðleggingar Landlæknis og "Diskins" um næringu sem að lágmarkar einning kostnaðinn. Við viljum því búa til forrit sem finnur ódýra og næringarríka kvöldmáltíð fyrir notendann. Við setjum upp bestunarverkefni með gurobipy pakkanum í python til þess að búa til þessa máltíð.



### 2. Líkan

Lýstu stærðfræðilega líkaninu sem þú notaðir. Útskýrðu vel allar ákvarðanabreytur, takmarkanir og markfall.

**Næringarþarfir**

Miðum næringarmarkmið við miðlungsvirkann karlmann á aldrinum 18-24 sem neðri mörk. Við byrjum á því að takmarka ekkert næringarfnin. Okkar markmið er að búa til máltíð sem inniheldur neðri mörk 40% af daglegum næringarjukki og efri mörk 50%. Prótein er 20 E% svo prótín = 2800 * 20/100 / 4 = 140. Kolvetni 55% og fita 25%. Erum að miða við fit manneskju. Setjum markmið að ná þessu, ekki jafn mikið
| | Miðlungsvirkur karlmaður 18-24 ára | 40%| 50% |
|----|----------|---------|-----|
| Orka | 2820  kcal     |  1128 kcal| 1410 kcal|
| Prótín  |  140g   | 56g| 70g |
| Kolvetni| 385g | 154g | 192.5g|
| Fita | 77.8g | 31.1g| 38.8g|
|Trefjar | 35g | 14g | 17.5g|
| A-vítamín |800 |320 | 400|



**Mengi og vísar**
- $j\in J$: $j$ er vísir í hráefni og $J$ mengi þeirra allra
- $i\in I$: $i$ er vísir í mengi $I$ = {Flokkur (prótein / kolvetni / grænmeti‐ávextir) ; Verð (kr.) / 100 g ; Vegan ; Orka (kcal) / 100 g ; Prótein (g) / 100 g ; Kolvetni (g) / 100 g ; Fita (g) / 100 g ; Trefjar (g) / 100 g ; A-vítamín (µg)}

**Fastar**
- $A$ er $|I|\times |J|$ fylki þar sem $A_{i,j}$ er gildi hráefnis $i$ í flokki $j$

**Ákvörðunarbreytur**
- $x$ er $|J|$ víður vigur þar sem $x_j$ táknar magn í 100g af hráefni $j$

**Markfall**
Lágmörkum kostnað máltíðarinnar:
$$
\text{min} \sum_{}
$$
þar sem $i$ er vísirinn sem dregur út röðina sem svarar til Verð (kr.) / 100 g.


**Skorður**

- Þrjú hráefni skulu vera valin
$$
|\{x_j | x_j\neq 0\}| = 3
$$
- Það skulu vera minnst ? kaloríur af hverjum flokki
$$
\sum 
$$


orka?

prótín = (orka*(E%/100)) / 4


***Byrjum á því að lesa inn og vinna úr gögnunum okkar***

In [59]:
import pandas as pd
import numpy as np

#Lesum inn gögn
df = pd.read_csv('ing.csv')

#Fjarlægjum tómar línur
df.dropna(inplace = True)

df['Flokkur (prótein / kolvetni / grænmeti‐ávextir)'] = (
    df['Flokkur (prótein / kolvetni / grænmeti‐ávextir)']
    .str.replace('‐', '-', regex=False)  # Breyta Unicode langa bandstrikinu í venjulegt bandstrik
    .str.strip()  # Fjarlægja auka bil
    .str.lower()  # Breyta í lágstafi
)

# Foods eru mengi vísa fyrir hráefnin í töflunni
foods = list(range(len(df.index)))

# A er fylki viðeigandi gilda þ.á.m. orka / 100g, prótein / 100g o.s.frv.
A = np.array(df.iloc[:,[1,2,3,4,5,6,7,8,9,10]])

# Breytum flokksbreytum í talnabreytur þannig að prótein er 0, kolvetni 1 og grænmeti-ávextir 2
flokkar = ['prótein', 'kolvetni','grænmeti-ávextir']
for i in range(3):
    A[A == flokkar[i]] = i

***Búum til líkanið***

In [60]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np


def bestum_rett(kyn, virkni, aldur, e_prosent, einkunn):
    """
    Býr til máltíð sem uppfyllir viðmiðunargildi Landlæknis og disksins fyrir manneskjur á aldrinum 18 og eldri

    Args:
        kyn (string): kvk fyrir kvennkyn og kk fyrir karlkyn.
        virkni (int): 0 fyrir litla virkni, 1 fyrir meðal virkni og 2 fyrir mikla virkni.
        aldur (int): aldur notenda
        e_prosent (array): vigur sem skilgreinir orkuprosentu
        einkunn (boolean): hvort einkunn sé tekin með í markfallið

    Returns:
        Prentar niðurstöður módelsins

    """

    #Viðmiðunargildi fyrir orkuþörf fullorðina
    orka = {'kvk-0': [1984,1912,1721,1697], 'kvk-1': [2247,2151,1984,1960], 'kvk-2': [2533,2438,2223,2199], 
            'kk-0': [2486,2366,2151,2103], 'kk-1': [2820,2701,2462,2414], 'kk-2': [3155,3035,2772,2701]}
    trefjar = {'kvk': 25, 'kk': 35}
    a_vit = {'kvk': 700, 'kk': 800} 

    if sum(e_prosent) != 100:
        print('Villa: Ógild E%% gildi')
        return None

    if aldur < 18:
        print('Villa: Manneskja yngri en 18')
        return None
    
    aldur_mork = [18,25,51,70]
    for i in range(len(aldur_mork) - 1):
        if aldur_mork[i] <= aldur < aldur_mork[i + 1]:
            orku_thorf = orka[f'{kyn}-{virkni}'][i]
    
    #Viðmiðunargildi fyrir notenda
    magn = {
        'orka': orku_thorf,
        'prótein': orku_thorf * (e_prosent[0] / 100) / 4,
        'kolvetni': orku_thorf * (e_prosent[1] / 100) / 4,
        'fita': orku_thorf * (e_prosent[2] / 100) / 9,
        'trefjar': trefjar[kyn],
        'a-vítamín': a_vit[kyn]
    }
    keys = ['orka', 'prótein','kolvetni','fita','trefjar','a-vítamín']
    
    #Búum til módel
    model = gp.Model("Dish")
    x = model.addVars(foods, ub = 2)
    
    #Setjum markfall, A[f,9] ekki tekið með ef einkunn = 0 (false)
    model.setObjective(gp.quicksum((A[f,1]+einkunn*(10-A[f,9])*100)*x[f] for f in foods), GRB.MINIMIZE) 

    # Jafn mikið af hverjum flokki
    model.addConstrs(
    (gp.quicksum(x[f] for f in foods) / 3 == gp.quicksum(x[f] for f in foods if A[f,0] == c)
     for c in [0,1,2])
    )

    # Minnsta kosti 50g af hverju hráefni

    # Máltíð þarf að uppfylla skilyrði landlæknis. Máltíð þriðjungur af inntöku þann daginn
    for i in range(3,9):
        model.addConstr(gp.quicksum(A[f,i]*x[f] for f in foods) == magn[keys[i-3]] / 3)

    #Bestum
    model.setParam('OutputFlag', 0)
    model.optimize()

    #Tökum saman niðurstöður
    xnp = np.array([x[f].X for f in foods])
    non_zero_indices = np.nonzero(xnp)[0]
    print("Hráefni máltíðar")
    for indice in non_zero_indices:
        print(f"{df.iloc[indice,0]}, {np.round(xnp[indice]*100,1)}g")

    print(f"\nKostnaður: {int(sum(xnp[f]*A[f,1] for f in foods))} kr\n")
    for c in [0,1,2]:
        print(f"Magn {flokkar[c]}: {np.round(sum(xnp[f] for f in foods if A[f,0] == c)*100,1)}g")

    print("\nNæringargildi")
    print(f"Orka: {np.round(sum(A[f,3]*xnp[f] for f in non_zero_indices))} kcal" )
    print(f"Prótein: {np.round(sum(A[f,4]*xnp[f] for f in non_zero_indices),1)}g" )
    print(f"Kolvetni: {np.round(sum(A[f,5]*xnp[f] for f in non_zero_indices),1)}g" )
    print(f"Fita: {np.round(sum(A[f,6]*xnp[f] for f in non_zero_indices),1)}g" )
    print(f"Trefjar: {np.round(sum(A[f,7]*xnp[f] for f in non_zero_indices),1)}g" )
    print(f"A vítamín: {np.round(sum(A[f,8]*xnp[f] for f in non_zero_indices),1)}μg" )


***Keyrum líkanið fyrir stakan notanda***

In [58]:
#Keyrum Bestum Rétt módelið fyrir notendann mig!

kyn = 'kk'
aldur = 22
virkni = 1
E = [20,55,25]

# Við sleppum einkunnum hér
bestum_rett(kyn,virkni,aldur,E,False)


Hráefni máltíðar
BIOTech USA próteinduft, 11.7g
Fiskibollur, 122.7g
Heilhveiti fusilli pasta, 148.6g
Jarðhnetur , 9.1g
Lambalifur, 4.1g
Maiskökur, 6.1g
Pastasósa classic, 145.7g
Rjómi, 16.3g

Kostnaður: 620 kr

Magn prótein: 154.8g
Magn kolvetni: 154.8g
Magn grænmeti-ávextir: 154.8g

Næringargildi
Orka: 940.0 kcal
Prótein: 47.0g
Kolvetni: 129.3g
Fita: 26.1g
Trefjar: 11.7g
A vítamín: 266.7μg
